In [1]:
import pandas as pd
import numpy as np
import os
import re
import ast
%config PPMagics.autolimit=0
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)


In [2]:
transcripts_features_WK36_Validated_stage1 = pd.read_csv('dailogue_savings_products_1000.csv')
transcripts_features_WK36_Validated_stage1.shape

(945, 5)

## phone

In [3]:
import re

NUMBER_WORDS = ["one", "two", "three", "four", "five", "six", "seven", "eight", "nine", "ten",  # units
                "eleven", "twelve", "thirteen", "fourteen", "fifteen", "sixteen", "seventeen", "eighteen", "nineteen",
                "twenty", "thirty", "forty", "fifty", "sixty", "seventy", "eighty", "ninety",  # tens
                "hundred", "thousand", "million", "billion", "trillion",
                "single", "double", "triple", "quadruple", "quintuple", "sextuple", "septuple", "octuple", "nonuple",
                "decuple"]

CURRENCY_WORDS = ["cents", "dollars", "pound", "quid"]

PAUSE_IDENTIFIER = "[pause]"
SCRUB_IDENTIFIER = "[scrub]"

# Reference: https://huggingface.co/transformers/model_doc/longformer.html#transformers.LongformerTokenizer ,
# https://huggingface.co/transformers/model_doc/roberta.html#transformers.RobertaTokenizer
SEP_TOKEN = "</s>"
CLS_TOKEN = "<s>"


def process_voice_raw_text(raw_text, minimial=False):
    raw_text = str(raw_text).lower()
    raw_text = raw_text.replace("\"", "")  # sometimes conversations are quoted in old scrubbed data
    sender_idxes = [m.span() for m in re.finditer(r'(agent:)|(customer:)|(unknown:)', raw_text)]

    # break into senders
    parsed_content = _separate_senders(sender_idxes, raw_text, minimial)
    return CLS_TOKEN + " " + (" " + SEP_TOKEN + " ").join(parsed_content) + " " + SEP_TOKEN

def _separate_senders(sender_idxes, raw_text, minimial):
    parsed_content = []
    for idx, (std, end) in enumerate(sender_idxes):
        sender = raw_text[std: end]
        c_std = end + 1
        c_end = sender_idxes[idx + 1][0] if idx < len(sender_idxes) - 1 else -1
        sender_content = raw_text[c_std:c_end] if c_end > 0 else raw_text[c_std:]
        sender_content = _voice_pre_process(sender_content, minimial).rstrip()
        parsed_content.append(" ".join([sender.upper(), sender_content]))
    return parsed_content

def _voice_pre_process(sub_sentence, minimial=False):
    # replace all digits
    digit_regex = re.compile(r'\b%s\b' % r'\b|\b'.join(map(re.escape, NUMBER_WORDS)))
    sub_sentence = digit_regex.sub("[digit]", sub_sentence)
    sub_sentence = re.sub(r'(?:\[digit\]\s*)*\[digit\]', '[digit]', sub_sentence)

    if not minimial:
        # voice separators
        sub_sentence = re.sub(r'[#]+', '[scrub]', sub_sentence)
        sub_sentence = re.sub(r'[.]+', PAUSE_IDENTIFIER, sub_sentence)

        # remove \t as these are present in old formats.
        sub_sentence = re.sub(r'<split>', ' ', sub_sentence)

        # remove repeated spaces
        sub_sentence.replace("  ", " ")

        # replace money, avoid since money can have country info.
        # sub_sentence = re.sub(r'\[digit\]\s*(quid|dollars)', '[money]', sub_sentence)

        # replace duplicated trigrams
        sub_sentence = re.sub(r'\b(\w+)\b\s*(\w+)\b\s*(\w+)(( \1\b)( \2\b)( \3\b))+', r'\1 \2 \3 [tri-gram-repeat]', sub_sentence)

        # replace duplicated bigrams
        sub_sentence = re.sub(r'\b(\w+)\b\s*(\w+)(( \1\b)( \2\b))+', r'\1 \2 [bi-gram-repeat]', sub_sentence)

        # replace duplicated words
        sub_sentence = re.sub(r'\b(\w+)( \1\b)+', r'\1 [repeat]', sub_sentence)
    return sub_sentence

In [4]:
transcripts_features_WK36_Validated_stage1_phone=transcripts_features_WK36_Validated_stage1[transcripts_features_WK36_Validated_stage1['media_type']=='phone'].reset_index(drop=True)
transcripts_features_WK36_Validated_stage1_phone.shape

(945, 5)

In [5]:
transcripts_features_WK36_Validated_stage1_phone['all_utterances_proc'] = transcripts_features_WK36_Validated_stage1_phone['all_utterances']
transcripts_features_WK36_Validated_stage1_phone['all_utterances_proc'] = transcripts_features_WK36_Validated_stage1_phone['all_utterances_proc'].apply(process_voice_raw_text)
transcripts_features_WK36_Validated_stage1_phone.shape


(945, 6)

In [6]:
transcripts_features_WK36_Validated_stage1_phone.head(1)[['all_utterances','all_utterances_proc']]

,all_utterances,all_utterances_proc
0,Customer: calling paypal this is my name may i have your first and last name please\tAgent: and about the right\tCustomer: hi how may i help you this ... afternoon\tAgent: oh i would just call in ah um ... i wanna transfer ... a ... little bit of money my paypal account i ... transfer it to my saved ... paypal savings account ... you tell me how to do that again\tCustomer: ok one moment\tAgent: ok\tCustomer: ok so do you see ... um in your savings account where it says ... money\tAgent: yeah\tCustomer: ok so you're gonna click that\tAgent: ok\tCustomer: and then you're gonna ... have money to your general savings\tAgent: or existing go ... yeah i know i i know all that i i wanna know how transfer from my paypal ... my paypal balance into the savings ... ok so ... that's not what was said\tCustomer: ok ... you would do the same way and they would ask you how ... um ... how you would like to add it ... and then you should be able to click ... your balance if you have money in your balance\tAgent: so that i had added money\tCustomer: yeah general\tAgent: ok ... so it popped up general statement and then it says ... choose how much ... add ... and that's all it shows me ... it doesn't have the yep ... spot where i ... choose ... ok ... we ... have\tCustomer: we got until the next page yeah\tAgent: yeah it says ... time today\tCustomer: so you only have sixteen dollars in your account ... so is it not ... it's not giving you the option to ... type in however much you want\tAgent: no i i wanna ... the last i just ... um ... i spoke to a lady last month i don't remember how to do it she told me how to do it ... um and i'm just trying to find out how to do it\tCustomer: ok one moment\tAgent: ok thank you\t,<s> CUSTOMER: calling paypal this is my name may i have your first and last name please </s> AGENT: and about the right </s> CUSTOMER: hi how may i help you this [pause] afternoon </s> AGENT: oh i would just call in ah um [pause] i wanna transfer [pause] a [pause] little bit of money my paypal account i [pause] transfer it to my saved [pause] paypal savings account [pause] you tell me how to do that again </s> CUSTOMER: ok [digit] moment </s> AGENT: ok </s> CUSTOMER: ok so do you see [pause] um in your savings account where it says [pause] money </s> AGENT: yeah </s> CUSTOMER: ok so you're gonna click that </s> AGENT: ok </s> CUSTOMER: and then you're gonna [pause] have money to your general savings </s> AGENT: or existing go [pause] yeah i know i [repeat] know all that i [repeat] wanna know how transfer from my paypal [pause] my paypal balance into the savings [pause] ok so [pause] that's not what was said </s> CUSTOMER: ok [pause] you would do the same way and they would ask you how [pause] um [pause] how you would like to add it [pause] and then you should be able to click [pause] your balance if you have money in your balance </s> AGENT: so that i had added money </s> CUSTOMER: yeah general </s> AGENT: ok [pause] so it popped up general statement and then it says [pause] choose how much [pause] add [pause] and that's all it shows me [pause] it doesn't have the yep [pause] spot where i [pause] choose [pause] ok [pause] we [pause] have </s> CUSTOMER: we got until the next page yeah </s> AGENT: yeah it says [pause] time today </s> CUSTOMER: so you only have [digit] dollars in your account [pause] so is it not [pause] it's not giving you the option to [pause] type in however much you want </s> AGENT: no i [repeat] wanna [pause] the last i just [pause] um [pause] i spoke to a lady last month i don't remember how to do it she told me how to do it [pause] um and i'm just trying to find out how to do it </s> CUSTOMER: ok [digit] moment </s> AGENT: ok thank you </s>


In [18]:
transcripts_features_WK36_Validated_stage1_phone.to_csv('preprocessed_daig_savings_proucts_1000.csv',index=False)

In [7]:
list(transcripts_features_WK36_Validated_stage1_phone)

['Case_ID',
 'media_type',
 'all_utterances',
 'translation_flag',
 'lang',
 'all_utterances_proc']

In [8]:
transcripts_features_WK36_Validated_stage1_phone.shape

(945, 6)

In [13]:
data_received=pd.read_excel(r'HYSA_1000 1st review 4th Sep.xlsx')[['Voice Id','HYSA Complaint Y_N']]
data_received=data_received[~data_received['HYSA Complaint Y_N'].isnull()].reset_index(drop=True)
data_received=data_received.drop_duplicates().reset_index(drop=True)
data_received.shape

(948, 2)

In [18]:
data_received[data_received['Voice Id'].isin([7230822024777634962,7231586060804890430,7231934623170764441])]



,Voice Id,HYSA Complaint Y_N
247,7231934623170764441,No
361,7230822024777634962,Yes
436,7231586060804890430,Yes
450,7230822024777634962,No
451,7231586060804890430,No
452,7231934623170764441,Yes


In [24]:
data_received=data_received.rename(columns={'Voice Id':'Case_ID'})

In [20]:
data_received=data_received[~data_received['Voice Id'].isin([7230822024777634962,7231586060804890430,7231934623170764441])].reset_index(drop=True)
data_received.shape

transcripts_features_WK36_Validated_stage1_phone=transcripts_features_WK36_Validated_stage1_phone[~transcripts_features_WK36_Validated_stage1_phone['Case_ID'].isin([7230822024777634962,7231586060804890430,7231934623170764441])].reset_index(drop=True)
transcripts_features_WK36_Validated_stage1_phone.shape

(942, 2)

In [23]:
len(set(transcripts_features_WK36_Validated_stage1_phone['Case_ID']) & set(data_received['Voice Id']))


942

In [25]:
final=pd.merge(transcripts_features_WK36_Validated_stage1_phone,data_received,on='Case_ID',how='left')



In [26]:
final.shape

(942, 7)

In [27]:
list(final)

['Case_ID',
 'media_type',
 'all_utterances',
 'translation_flag',
 'lang',
 'all_utterances_proc',
 'HYSA Complaint Y_N']

In [29]:
final['HYSA Complaint Y_N'].value_counts()

No     558
Yes    384
Name: HYSA Complaint Y_N, dtype: int64

In [31]:
final.head(1)

,Case_ID,media_type,all_utterances,translation_flag,lang,all_utterances_proc,HYSA Complaint Y_N
0,7228301746558280648,phone,Customer: calling paypal this is my name may i have your first and last name please\tAgent: and about the right\tCustomer: hi how may i help you this ... afternoon\tAgent: oh i would just call in ah um ... i wanna transfer ... a ... little bit of money my paypal account i ... transfer it to my saved ... paypal savings account ... you tell me how to do that again\tCustomer: ok one moment\tAgent: ok\tCustomer: ok so do you see ... um in your savings account where it says ... money\tAgent: yeah\tCustomer: ok so you're gonna click that\tAgent: ok\tCustomer: and then you're gonna ... have money to your general savings\tAgent: or existing go ... yeah i know i i know all that i i wanna know how transfer from my paypal ... my paypal balance into the savings ... ok so ... that's not what was said\tCustomer: ok ... you would do the same way and they would ask you how ... um ... how you would like to add it ... and then you should be able to click ... your balance if you have money in your balance\tAgent: so that i had added money\tCustomer: yeah general\tAgent: ok ... so it popped up general statement and then it says ... choose how much ... add ... and that's all it shows me ... it doesn't have the yep ... spot where i ... choose ... ok ... we ... have\tCustomer: we got until the next page yeah\tAgent: yeah it says ... time today\tCustomer: so you only have sixteen dollars in your account ... so is it not ... it's not giving you the option to ... type in however much you want\tAgent: no i i wanna ... the last i just ... um ... i spoke to a lady last month i don't remember how to do it she told me how to do it ... um and i'm just trying to find out how to do it\tCustomer: ok one moment\tAgent: ok thank you\t,Y,en,<s> CUSTOMER: calling paypal this is my name may i have your first and last name please </s> AGENT: and about the right </s> CUSTOMER: hi how may i help you this [pause] afternoon </s> AGENT: oh i would just call in ah um [pause] i wanna transfer [pause] a [pause] little bit of money my paypal account i [pause] transfer it to my saved [pause] paypal savings account [pause] you tell me how to do that again </s> CUSTOMER: ok [digit] moment </s> AGENT: ok </s> CUSTOMER: ok so do you see [pause] um in your savings account where it says [pause] money </s> AGENT: yeah </s> CUSTOMER: ok so you're gonna click that </s> AGENT: ok </s> CUSTOMER: and then you're gonna [pause] have money to your general savings </s> AGENT: or existing go [pause] yeah i know i [repeat] know all that i [repeat] wanna know how transfer from my paypal [pause] my paypal balance into the savings [pause] ok so [pause] that's not what was said </s> CUSTOMER: ok [pause] you would do the same way and they would ask you how [pause] um [pause] how you would like to add it [pause] and then you should be able to click [pause] your balance if you have money in your balance </s> AGENT: so that i had added money </s> CUSTOMER: yeah general </s> AGENT: ok [pause] so it popped up general statement and then it says [pause] choose how much [pause] add [pause] and that's all it shows me [pause] it doesn't have the yep [pause] spot where i [pause] choose [pause] ok [pause] we [pause] have </s> CUSTOMER: we got until the next page yeah </s> AGENT: yeah it says [pause] time today </s> CUSTOMER: so you only have [digit] dollars in your account [pause] so is it not [pause] it's not giving you the option to [pause] type in however much you want </s> AGENT: no i [repeat] wanna [pause] the last i just [pause] um [pause] i spoke to a lady last month i don't remember how to do it she told me how to do it [pause] um and i'm just trying to find out how to do it </s> CUSTOMER: ok [digit] moment </s> AGENT: ok thank you </s>,No


In [32]:
final.to_csv('preprocessed_daig_savings_proucts_1000.csv',index=False)